In [33]:
import numpy as np


In [34]:
sigma = np.float64(5.67*10**(-8))
earth_radius = np.float64(6371000)
earth_mass = np.float64(5.976 * 10**24)
G = np.float64(6.67 * 10**(-11))

As = np.array([0.6] * 6) #
e = np.array([0.4] * 6) #
qs = np.float64(1367)
generationHeat = np.array([50] * 6)

width = 0.005
Lx = np.float64(1)
Ly = np.float64(1)
Lz = np.float64(1)
R_value = 2

c = 800
p = 2700

orbit_radius = np.float64(500)


In [35]:
Qav = np.zeros(6)
Qav += generationHeat
Qav

orbit_radius *= 1000
angleSunSet = np.arccos(earth_radius/(earth_radius+orbit_radius))
sigma = np.float64(5.67*10**(-8))

areas = np.array([Ly*Lz, Ly*Lz,
                Lx*Lz, Lx*Lz,
                Lx*Ly, Lx*Ly])

Qav[0] += areas[0] * As[0] * qs/np.pi #зенит
Qav[5] += areas[5] * As[5] * qs/np.pi #солнечная до 6 часов
Qav[1] += np.sin(angleSunSet) * areas[1] * As[1] * qs/np.pi #надир после 6 часов
Qav[5] += (1 - np.cos(angleSunSet)) * areas[5] * As[5] * qs/np.pi #после до 6 часов

Qav


array([311.07776865, 147.77137832,  50.        ,  50.        ,
        50.        , 330.07629642])

In [36]:
Qex = np.zeros(6)
Qex[0] += qs * As[0] * areas[0] * np.sqrt(2)/2
Qex[1] += qs * As[1] * areas[1] * np.sqrt(2)/2
Qex[5] += qs * As[5] * areas[5] * np.sqrt(2)/2

Qex += generationHeat
Qex


array([629.96898193, 629.96898193,  50.        ,  50.        ,
        50.        , 629.96898193])

In [37]:
Qsum = np.sum(Qav)

radiationCoef = np.sum(e[:]*areas[:]*areas[:]*sigma)
Tav = (Qsum/radiationCoef)**0.25
Tav


288.2101037950334

In [38]:
R = R_value * np.ones(shape=(6,6))
for i in range(6):
    R[i][i] = 10**24
for i in range(3):
    R[2*i+1][2*i] = 10**24
    R[2*i][2*i+1] = 10**24
R


array([[1.e+24, 1.e+24, 2.e+00, 2.e+00, 2.e+00, 2.e+00],
       [1.e+24, 1.e+24, 2.e+00, 2.e+00, 2.e+00, 2.e+00],
       [2.e+00, 2.e+00, 1.e+24, 1.e+24, 2.e+00, 2.e+00],
       [2.e+00, 2.e+00, 1.e+24, 1.e+24, 2.e+00, 2.e+00],
       [2.e+00, 2.e+00, 2.e+00, 2.e+00, 1.e+24, 1.e+24],
       [2.e+00, 2.e+00, 2.e+00, 2.e+00, 1.e+24, 1.e+24]])

In [39]:
mass = p*width*areas[:]
mass


array([13.5, 13.5, 13.5, 13.5, 13.5, 13.5])

In [40]:
radius = earth_radius*1000 + earth_radius # в метры
velocity = np.sqrt(G * earth_mass / radius) # скорость 
length = 2*np.pi*radius #длина орбиты
period = length/velocity #период полного оборота

tDay = period * (np.pi/2+angleSunSet)/(2*np.pi)


In [41]:
m = 1/2/R
for i in range(6):
    m[i][i] = np.sum(1/R/2) + c*mass[i]/tDay
m


array([[6.0002166e+00, 5.0000000e-25, 2.5000000e-01, 2.5000000e-01,
        2.5000000e-01, 2.5000000e-01],
       [5.0000000e-25, 6.0002166e+00, 2.5000000e-01, 2.5000000e-01,
        2.5000000e-01, 2.5000000e-01],
       [2.5000000e-01, 2.5000000e-01, 6.0002166e+00, 5.0000000e-25,
        2.5000000e-01, 2.5000000e-01],
       [2.5000000e-01, 2.5000000e-01, 5.0000000e-25, 6.0002166e+00,
        2.5000000e-01, 2.5000000e-01],
       [2.5000000e-01, 2.5000000e-01, 2.5000000e-01, 2.5000000e-01,
        6.0002166e+00, 5.0000000e-25],
       [2.5000000e-01, 2.5000000e-01, 2.5000000e-01, 2.5000000e-01,
        5.0000000e-25, 6.0002166e+00]])

In [42]:
d = Qex[:] - Qav[:] + Tav*c*mass[:]/tDay
d


array([3.18953640e+02, 4.82260030e+02, 6.24267690e-02, 6.24267690e-02,
       6.24267690e-02, 2.99955112e+02])

In [43]:
from scipy import linalg
linalg.solve(m, d) + Tav


array([340.28530739, 367.50205657, 281.07029384, 281.07029384,
       283.34203664, 333.32234659])

In [48]:
Qex = generationHeat
Qex


array([50, 50, 50, 50, 50, 50])

In [49]:
tNight = period - tDay

m = 1/2/R
for i in range(6):
    m[i][i] = np.sum(1/R/2) + c*mass[i]/tNight

d = Qex[:] - Qav[:] + Tav*c*mass[:]/tNight


In [51]:
from scipy import linalg
linalg.solve(m, d) + Tav


array([246.13215207, 273.3494401 , 292.36286566, 292.36286566,
       290.24114786, 243.56252607])